In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml

In [2]:
df = pd.read_csv("data.csv", header=None, index_col=0)
df

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,,,,,,,,,,,,,,,,,,,,,
842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820


In [3]:
df = df.fillna(df.mean())
df

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,,,,,,,,,,,,,,,,,,,,,
842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820


In [4]:
labels = df.pop(1).eq('M').mul(1)
data = (df - df.mean()) / df.std()

In [5]:
X, y = data.values.astype(float), labels.values.astype(float)

In [6]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [52]:
class NNLayer:
    def __init__(self, s_in, s_out, isout=False, learning_rate=0.01):
        self.s_in = s_in
        self.s_out = s_out
        self.W = np.random.randn(s_in, s_out)
        self.b = np.zeros(s_out)
    
    def forward(self, X):
        self.output = sigmoid(X.dot(self.W) + self.b)
        return self.output
    
    def compute_error(self, X, y):
        if self.isout:
            return self.output - y
        
    
class NeuralNet:
    def __init__(self, depth, s_in, s_out, learning_rate=0.01):
        self.layers = []
        self.depth = depth
        decr = s_in // nb_layers
        out = s_in
        for i in range(depth - 1):
            self.layers.append(NNLayer(s_in, out, learning_rate=learning_rate))
            s_in, out = out, s_in - decr
        self.layers.append(NNLayer(s_in, s_out, isout=True, learning_rate=learning_rate))
    
    def forward(self, X):
        a = X
        for layer in self.layers:
            a = layer.forward(a)
        return a
    
    def backward(self, X, y):
        for layer in reversed(self.layers):
            layer.compute_error(X, y)

In [53]:
network = NeuralNet(3, X.shape[1], 1)

In [54]:
network.forward(X) - 

array([[0.5844285 ],
       [0.92673264],
       [0.428762  ],
       [0.51402121],
       [0.86819377],
       [0.42723541],
       [0.86376084],
       [0.73774295],
       [0.31684357],
       [0.90061482],
       [0.94478249],
       [0.72341956],
       [0.38652225],
       [0.58676237],
       [0.64250715],
       [0.86860288],
       [0.94523265],
       [0.43458456],
       [0.6555051 ],
       [0.89696467],
       [0.91505977],
       [0.89620043],
       [0.47307553],
       [0.73480576],
       [0.47118629],
       [0.4676161 ],
       [0.66397336],
       [0.79279814],
       [0.46866656],
       [0.93647286],
       [0.30128781],
       [0.80040154],
       [0.35058186],
       [0.44517482],
       [0.71828254],
       [0.44182568],
       [0.72166815],
       [0.948102  ],
       [0.85284954],
       [0.79233361],
       [0.92451986],
       [0.62711598],
       [0.43976627],
       [0.71180661],
       [0.72902206],
       [0.48559754],
       [0.91756011],
       [0.606